In [352]:
import numpy as np
import numpy.testing as nptests
import scipy as sp
import matplotlib as mlp
import matplotlib.pyplot as plt

import scipy.integrate as integrate

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
from collections import OrderedDict
from functools import reduce

import unittest
import copy

In [318]:
plt.style.use('dark_background')

In [364]:
class massActionCRN( object ):
    
    def __init__( self, crn_json_name, _debug = False ):
        
        self._debug = _debug
        
        with open(crn_json_name, "r") as f:
            self._crn_dict = json.load( f, object_pairs_hook=OrderedDict )
        
        self._x0 = self._get_x0_from_json( self._crn_dict['species'] )
        self._check_x0()
        
        self._rates = self._get_rates( self._crn_dict['reactions'] )
        
        self._reaction_list = self._get_reactions( self._crn_dict['reactions'] )
        self._gamma = self._get_gamma( self._reaction_list )
        
        self._left_side_matrix = np.vstack( [ r[ 0 ] for r in self._reaction_list ] )
        
        if self._debug:
            #print("json crn dict:", self._crn_dict)
            print( "json crn dict:", json.dumps(self._crn_dict, indent=4))
            print( "x0:", self._x0 )
            print( "rates:", self._rates )
            print("gamma:", self._gamma)
        
    def get_species( self ):
        """ Returns the list of species in order.
        """
        return list( self._crn_dict['species'].keys() )
    
    def set_x0( self, new_x0 ):
        self._x0 = new_x0
        self._check_x0()
    
    def get_x0( self ):
        return self._x0
    
    def _check_x0( self ):
        """ Check that x0 is corresponding to relative concentrations.
        """
        if not np.isclose( self._x0.sum(), 1.0 ):
            raise TypeError("The CRN initial relative concentrations do not sum to 1: {}.".format(self._x0))
    
    def integrate( self ):
        time = np.linspace(0, 20000, 10000)
        return integrate.odeint( self._dynamic_map(), self._x0, time )
    
    def _dynamic_map( self ):
        def f(x, t0):
            speed_vector = (x**abs(self._left_side_matrix)).prod(axis = 1)*self._rates
            return self._gamma.dot( speed_vector )        
        return f
    
    def _get_x0_from_json( self, species_dict ):
        """
            species_dict: json ordered dict of species e.g. OrderedDict([('X', '0.5'), ('Y', '0.25'), ('Z', '0.25')]))

            Returns the numpy vector of initial relative concentrations e.g. np.array([0.5, 0.25, 0.25])
        """
        return np.array( list( map( float, species_dict.values() ) ) )
    
    def _get_rates( self, reactions_dict ):
        """
            reactions_dict: json ordered dict of reactions e.g. OrderedDict([('2X + 3Y <-> Z', ['0.110', '0.0008']), ('Z  + 5X  -> Y', ['0.04'])])

            Returns the numpy vector of reaction rates e.g. np.array([0.11  , 0.0008, 0.04  ])
        """
        
        
        return np.array( list ( map( float, reduce( lambda a,b : a+b, reactions_dict.values() ) ) ) )
    
    def _get_gamma( self, reaction_list ):
        """
            reaction_list: a list of reaction as returned by _get_reactions
            
            Returns the stochiometric matrix associated to the CRN
        """
        return np.array( list( map( lambda x: reduce(lambda a,b: a+b, x), reaction_list ) ) ).T
    
    def _get_reactions( self, reactions_dict ):
        """
            reactions_dict: json ordered dict of reactions e.g. OrderedDict([('2X + 3Y <-> Z', ['0.110', '0.0008']), ('Z  + 5X  -> Y', ['0.04'])])
            
            Returns a list of reactions where a reaction is list with two element:
                - The first element is the stochiometrie array of each species as reactants of the reaction (negative or 0)
                - The second element is the stochiometrie array of each species as products of the reaction (positive or 0)
                
            e.g. on the example above: 
            
            species: X, Y, Z
            [[array([-2., -3.,  0.]), array([0., 0., 1.])],
             [array([-0., -0., -1.]), array([ 2.,  3., -0.])],
             [array([-5.,  0., -1.]), array([0., 1., 0.])]]
        """
        reaction_list = []
        
        for r in reactions_dict:
            
            clean_r = r.replace(" ", "")
            
            if '<->' in r:
                separator = '<->'
            else:
                separator = '->'
            
            central_split = clean_r.split( separator )
            
            reaction = []
            for i_side, side in enumerate( central_split ):
                species_split = side.split("+")
                
                reaction_side = np.zeros( len( self._x0 ) )
                
                if side != '':
                    for s in species_split:
                        name, coeff = self._get_specie_and_stoc( s )
                        index_in_reaction_side = list( self._crn_dict[ 'species' ] ).index( name )
                        reaction_side[ index_in_reaction_side ] = -1*coeff if i_side == 0 else coeff
                
                reaction.append( reaction_side )
            
            reaction_list.append( reaction )
            
            if separator == '<->':
                reaction_list.append( [ -1*reaction[ 1 ], -1*reaction[ 0 ] ] )
                
        return reaction_list
    
    def _get_specie_and_stoc( self, specie_in_reac ):
        """
            specie_in_reac: string of the form <float><specie_name> e.g.: 2X, 2.89Y_0
            
            Warning: the function will raise exception if:
                - no specie is found in the string e.g.: '', '234'
                - <specie_name> was not listed in the list of species
            
            Returns the couple: <specie_name>, <float>
        """
        
        regex_first_letter = r"[a-zA-Z]"
        matches = re.finditer(regex_first_letter, specie_in_reac)
        

        i_first_letter = -1
        for match in matches:
            i_first_letter = match.start()
            break
            
        if i_first_letter == -1:
            raise TypeError("No specie found in reactant/product {}.".format( specie_in_reac ) )
        
        specie_name = specie_in_reac[i_first_letter:]
        
        if specie_name not in self._crn_dict['species']:
            raise TypeError("Reaction mentions specie {} which was not found in species list.".format( specie_name ) )
        
        specie_coeff_str = specie_in_reac[:i_first_letter]
        
        if specie_coeff_str == '':
            specie_coeff = 1
        else:
            specie_coeff = float( specie_coeff_str )
            
        return specie_name, specie_coeff
        
    
    
    def __str__( self ):
        pretty_str = ""
        
        for s in self._crn_dict['species']:
            pretty_str += s+"_0:" + " " + self._crn_dict['species'][ s ] + " "
        pretty_str += "\n"
        #pretty_str += str( self._crn_dict['species'] )+"\n"
        #pretty_str += "rates: " + str( self.rates )+"\nsystem:\n"
        for i,r in enumerate( self._crn_dict['reactions'] ):
            pretty_str += r + " " + str( list( map( float, self._crn_dict['reactions'][ r ] ) ) )
            
            if i+1 != len( self._crn_dict['reactions'] ):
                pretty_str += "\n"
        return pretty_str

In [365]:
crn1 = massActionCRN( "crn1.json" )

In [433]:
def build_CRN_ui( crn, manual = True ):
    
    def get_x0_from_dict( dict_ ):
        x0 = []
        for specie in crn.get_species():
            x0.append( float( dict_[ specie+"_0" ] ) )
        return np.array( x0 )
    
    def x0_to_dict( x0 ):
        dict_0 = {}
        for i, specie in enumerate( crn.get_species() ):
            dict_0[ specie+"_0" ] = str( x0[ i ] )
        return dict_0
    
    
    system_label = widgets.Textarea( str( crn ), disabled = True, rows = 1 + len(crn._crn_dict['reactions']) )
    
    def func( **kwargs ):
        new_crn = copy.deepcopy( crn )        
        history = new_crn.integrate()
        
        plt.figure( figsize = ( 20, 10 ) )
        
        for i, specie in enumerate( crn._crn_dict['species'] ):
            plt.plot( history[ :, i ], label = specie )
        
        plt.legend()
        plt.show()
    

    
    w = interactive(func, {'manual': manual}, **{} )
    
    
    
    display( widgets.VBox( [ w, widgets.Label("System:"), system_label ] ) )
    #display( widgets.VBox( [ w.children[ -1 ] ] + list( w.children[ : -1 ] )  ) )

In [434]:
build_CRN_ui( crn1, False )

In [423]:
?widgets.Textarea

Init signature: widgets.Textarea(*args, **kwargs)
Docstring:      Multiline text area widget.
Init docstring: Public constructor
File:           /nix/store/fz2djcck9v6681qxyi1yrax7bha8gmgy-python3.6-ipywidgets-7.4.2/lib/python3.6/site-packages/ipywidgets/widgets/widget_string.py
Type:           MetaHasTraits
